In [ ]:
import random
import heapq
from typing import List, Tuple
import itertools
import tracemalloc
from time import perf_counter_ns


class NQueens:
    """
    Class representing the N-Queens problem.
    """

    def __init__(self, n: int) -> None:
        self.n = n
        self.initial_state = tuple(random.randint(0, self.n-1)
                                   for _ in range(n))

    def actions(self, state: Tuple[int, ...]) -> List[Tuple[int, int]]:
        """
        Returns a list of valid actions that can be taken from the given state.
        """
        return [(i, j) for i in range(self.n) for j in range(self.n) if i != j]

    def result(self, state: Tuple[int, ...], action: Tuple[int, int]) -> Tuple[int, ...]:
        """
        Returns the new state resulting from taking the given action from the given state.
        """
        i, j = action
        new_state = list(state)
        new_state[i] = j
        return tuple(new_state)

    def is_goal_state(self, state: Tuple[int, ...]) -> bool:
        """
        Returns True if the given state is a goal state, False otherwise.
        """
        for i in range(self.n):
            for j in range(i + 1, self.n):
                if state[i] == state[j] or abs(state[i] - state[j]) == j - i:
                    return False
        return True

    def g(self, parent_cost: int, parent_state: Tuple[int, ...], action: Tuple[int, int], child_state: Tuple[int, ...]) -> int:
        """
        Returns the cost of the given child state, given the cost of the parent state and the action that was taken to get to the child state.
        """
        return parent_cost + 1

    def h(self, state: Tuple[int, ...]) -> int:
        """
        Returns the heuristic value of the given state.
        """

        columns = [0] * self.n
        left_diag = [0] * (2 * self.n - 1)
        right_diag = [0] * (2 * self.n - 1)
        count = 0

        for i in range(self.n):
            col = state[i]
            columns[col] += 1
            left_diag[i + col] += 1
            right_diag[self.n - 1 - i + col] += 1

        for i in range(self.n):
            col = state[i]
            count += (columns[col] - 1)
            count += (left_diag[i + col] - 1)
            count += (right_diag[self.n - 1 - i + col] - 1)
        return count

    # Another VERSION
    # def h(self, state: Tuple[int, ...]) -> int:
    #     """
    #     Returns the heuristic value of the given state.
    #     """
    #     columns = [0] * self.n
    #     left_diag = [0] * (2 * self.n - 1)
    #     right_diag = [0] * (2 * self.n - 1)
    #     count = 0

    #     for i in range(self.n):
    #         col = state[i]
    #         columns[col] += 1
    #         left_diag[i + col] += 1  # update left diagonal

    #     for i in range(self.n):
    #         col = state[i]
    #         right_diag[self.n - 1 - i + col] += 1  # update right diagonal

    #     for i in range(2 * self.n - 1):
    #         count += left_diag[i] * (left_diag[i] - 1) // 2  # count left diagonal conflicts
    #         count += right_diag[i] * (right_diag[i] - 1) // 2  # count right diagonal conflicts

    #     for i in range(self.n):
    #         count += (columns[i] * (columns[i] - 1)) // 2  # count column conflicts

    #     return count

    def f(self, node):
        """
        Returns the f value of the given state.
        """
        return node.g + self.h(node.state)


class NodeA:

    def __init__(self, problem: NQueens, state, g=0):
        self.problem = problem
        self.state = state
        self.g = g

    def __lt__(self, other):
        return self.problem.f(self) < self.problem.f(other)

    def print_queens(self):
        print(self.state)
        board = [["." for _ in range(len(self.state))]
                 for _ in range(len(self.state))]
        for i, j in enumerate(self.state):
            board[j][i] = "Q"
        for row in board:
            print(" ".join(row))


class AStarMinConflicts:

    def __init__(self, problem):
        """Initialize a A* solver for the N-Queens problem."""
        self.problem = problem
        self.root = NodeA(problem, problem.initial_state)
        self.goal_node = None

    def solve(self):
        # Initialize frontier with root node
        frontier = [self.root]
        heapq.heapify(frontier)

        # Initialize explored set with root node state
        explored = set()
        explored.add(self.root.state)

        # Main search loop
        while frontier:
            # Get node with lowest priority from frontier
            cur = heapq.heappop(frontier)

            # Check if current node is goal state
            if self.problem.is_goal_state(cur.state):
                self.goal_node = cur
                return cur

            # Generate child nodes and add to frontier if not already explored
            for action in self.problem.actions(cur.state):
                children_state = self.problem.result(cur.state, action)
                children = NodeA(self.problem, children_state, cur.g+1)
                if children.state not in explored:
                    heapq.heappush(frontier, children)
                    explored.add(children.state)
        # If goal state not found, return None
        return None


class NodeU:

    def __init__(self, problem: NQueens, state, g=0):
        self.problem = problem
        self.state = state
        self.g = g

    def __lt__(self, other):
        return self.g < other.g

    def print_queens(self):
        print(self.state)
        board = [["." for _ in range(len(self.state))]
                 for _ in range(len(self.state))]
        for i, j in enumerate(self.state):
            board[j][i] = "Q"
        for row in board:
            print(" ".join(row))


class UCS:

    def __init__(self, problem: NQueens):
        """Initialize a UCS solver for the N-Queens problem."""
        self.problem = problem
        self.root = NodeU(problem, problem.initial_state)
        self.goal_node = None

    def solve(self):
        # Initialize frontier with root node
        frontier = [self.root]
        heapq.heapify(frontier)

        # Initialize explored set with root node state
        explored = set()
        explored.add(self.root.state)

        # Main search loop
        while frontier:
            # Get node with lowest priority from frontier
            cur = heapq.heappop(frontier)

            # Check if current node is goal state
            if self.problem.is_goal_state(cur.state):
                self.goal_node = cur
                return cur

            # Generate child nodes and add to frontier if not already explored
            for action in self.problem.actions(cur.state):
                children_state = self.problem.result(cur.state, action)
                children = NodeU(self.problem, children_state, cur.problem.g(
                    cur.g, cur.state, action, children_state))
                if children.state not in explored:
                    heapq.heappush(frontier, children)
                    explored.add(children.state)

        # If goal state not found, return None
        return None


def fitness(state, n) -> int:
    """Returns the number of attacking queens."""
    columns = [0] * n
    left_diag = [0] * (2 * n - 1)
    right_diag = [0] * (2 * n - 1)
    count = 0

    for i in range(n):
        col = state[i]
        columns[col] += 1
        left_diag[i + col] += 1
        right_diag[n - 1 - i + col] += 1

    for i in range(n):
        col = state[i]
        count += (columns[col] - 1)
        count += (left_diag[i + col] - 1)
        count += (right_diag[n - 1 - i + col] - 1)
    return count


class GeneticAlgorithm:
    MUTATION_PROBABILITY: float = 0.95

    def __init__(self, problem: NQueens):
        """Initialize a genetic algorithm solver for the N-Queens problem."""
        self.problem = problem
        self.n = problem.n
        self.population_size = 2*self.n
        self.mutation_prob = GeneticAlgorithm.MUTATION_PROBABILITY
        self.selection_range = self.population_size // 4
        self.max_fitness = (self.n * (self.n - 1)) // 2
        self.half_queens = self.n//2
        self.new_generation = [-1]*(self.n + 1)
        self.gen = 0

        # Generate a random initial population
        self.population = list()
        for _ in range(self.population_size):
            chromosome = list(range(0, self.n))
            random.shuffle(chromosome)
            fitness_val = fitness(chromosome, self.n)
            chromosome.append(fitness_val)
            self.population.append(chromosome)
        self.population.sort(key=lambda chromosome: chromosome[-1])

    def selection(self):
        parent = (list(), self.n)
        while parent[1] == self.n:
            for _ in range(self.selection_range):
                random_chrom = random.choice(self.population)
                if random_chrom[-1] < parent[1]:
                    parent = (random_chrom, random_chrom[-1])
        return parent[0]
    # def selection(self):
    #     best_parent = None
    #     while best_parent is None or best_parent[-1] == self.n:
    #         for _ in range(self.selection_range):
    #             random_chrom = random.choice(self.population)
    #             if best_parent is None or random_chrom[-1] < best_parent[-1]:
    #                 best_parent = random_chrom
    #     return best_parent[:-1]

    def crossover(self, parent1, parent2):
        """Crossover two chromosome to produce offspring."""
        children = list()
        for _ in range(2):
            new_gen = self.new_generation

            index1, index2 = (random.randint(1, self.half_queens - 1), random.randint(self.half_queens + 1, self.half_queens + 1)) if self.n - \
                2 < self.half_queens + 1 else (random.randint(1, self.half_queens - 1), random.randint(self.half_queens + 1, self.n - 2))

            new_gen[index1: index2+1] = parent1[index1: index2+1]
            # for i in range(index1, index2+1):
            #     new_gen[i] = parent1[i]

            for i in range(index1, index2+1):
                if parent2[i] not in new_gen:
                    t = i
                    while index1 <= t <= index2:
                        t = parent2.index(parent1[t])
                    new_gen[t] = parent2[i]
            for j in range(self.n):
                if new_gen[j] == -1:
                    new_gen[j] = parent2[j]
            new_gen[-1] = fitness(new_gen, self.n)
            children.append(new_gen)
            parent1, parent2 = parent2, parent1
        return children

    def mutation(self, chromosome):
        """Mutate the chromosome to produce offspring."""
        index1, index2 = random.randint(
            0, self.n - 1), random.randint(0, self.n - 1)
        #index1, index2 = random.sample(range(self.n), 2)
        chromosome[index1], chromosome[index2] = chromosome[index2], chromosome[index1]
        chromosome[-1] = fitness(chromosome, self.n)

    # Another VERSION
    # def mutation(self, chromosome, min_swath_size=0.01, max_swath_size=0.53):
    #     """Mutate a chromosome (list) by applying the scramble mutation operator."""
    #     if random.random() < self.mutation_prob:
    #         size = len(chromosome)
    #         rand_swath_size = random.randint(
    #             int(min_swath_size*size), int(max_swath_size*size))
    #         if rand_swath_size < 2:
    #             rand_swath_size = 2
    #         start_idx = random.randint(0, size - rand_swath_size)
    #         subset = chromosome[start_idx:start_idx+rand_swath_size]
    #         if rand_swath_size <= 2:
    #             subset[0], subset[1] = subset[1], subset[0]
    #         else:
    #             random.shuffle(subset[1:-1])
    #         chromosome[start_idx:start_idx+rand_swath_size] = subset
    #     return chromosome

    def print_queens(self):
        """Print the board with the positions of the queens."""
        print(f'POPULATION SIZE: {self.population_size}')
        print(f'MUTATION PROBABILITY: {GeneticAlgorithm.MUTATION_PROBABILITY}')
        print(f"=== GENERATION {self.gen} ===")
        print(self.population[0])

        # Print the board
        board = ""
        for i, j in enumerate(self.population[0]):
            row = ["Q" if k == j else "." for k in range(
                len(self.population[0]))]
            board += " ".join(row) + "\n"

        print(board.strip())

    def evolve(self):
        """Evolve the population for a certain number of generations."""
        while self.population[1][-1] != 0:
            random.shuffle(self.population)
            new_gen = list()
            for _ in range(self.population_size//3):
                # Selection
                parent1, parent2 = self.selection(), self.selection()
                breed = False
                # Crossover
                children = self.crossover(parent1, parent2) if random.random() < 0.5 else [
                    parent1[:], parent2[:]]
                breed = True if random.random() < 0.5 else False
                # Mutation
                for child in children:
                    if random.random() < self.mutation_prob or not breed:
                        self.mutation(child)
                    new_gen.append(child)
            self.population.extend(new_gen)

            # Update population and generation
            self.population.sort(key=lambda x: x[-1])
            del self.population[self.population_size:]
            self.gen += 1

        del self.population[0][-1]


if __name__ == '__main__':
    N = int(input(f'Enter Number of Queens: '))  # the number of queens
    # 1: uniform-cost search, 2: A* with MIN-CONFLICT heuristic, 3: genetic algorithm
    print(f'1: Uniform-cost search, 2: A* with MIN-CONFLICT heuristic, 3: Genetic algorithm')
    algorithm = int(input(f'Enter Algorithm: '))
    timeAvg = []
    memoryAvg = []
    for i in range(3):
        tracemalloc.start()  # "get memory" start here
        time_start = perf_counter_ns()  # "get time" start here
        if algorithm == 1:
            ucs = UCS(NQueens(N))
            ucs.solve().print_queens()
        elif algorithm == 2:
            astar = AStarMinConflicts(NQueens(N))
            astar.solve().print_queens()
        elif algorithm == 3:
            genetic = GeneticAlgorithm(NQueens(N))
            genetic.evolve()
            genetic.print_queens()
        time_end = perf_counter_ns()  # "get time" end here
        memory = tracemalloc.get_traced_memory()[1]  # "get memory" end here
        tracemalloc.stop()
        timeAvg.append((time_end - time_start) / 10**6)
        memoryAvg.append(memory / 1024**2)

    print(f'Running time (average of 3 runs): {sum(timeAvg) / 3:.2f} ms.')
    print(f'Consumed memory (average of 3 runs): {sum(memoryAvg) / 3:.4f} MB.')


Enter Number of Queens: 100
1: Uniform-cost search, 2: A* with MIN-CONFLICT heuristic, 3: Genetic algorithm
Enter Algorithm: 3
POPULATION SIZE: 200
MUTATION PROBABILITY: 0.95
=== GENERATION 95 ===
[75, 13, 97, 77, 56, 23, 29, 60, 54, 36, 82, 53, 4, 73, 49, 43, 55, 81, 50, 19, 30, 99, 20, 26, 72, 79, 59, 38, 11, 88, 16, 57, 5, 0, 40, 9, 58, 87, 96, 15, 89, 70, 14, 27, 51, 10, 76, 68, 18, 24, 2, 46, 95, 98, 7, 71, 93, 74, 92, 28, 85, 80, 21, 45, 41, 67, 6, 35, 48, 66, 63, 91, 17, 33, 3, 39, 37, 65, 32, 64, 84, 1, 90, 61, 83, 94, 52, 34, 78, 47, 69, 12, 8, 44, 25, 86, 22, 31, 42, 62]
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . Q . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . 